In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, roc_curve, auc,
                             classification_report)
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
df1 = pd.read_csv('https://lms.skillfactory.ru/asset-v1:SkillFactory+MIPTDS+SEPT22+type@asset+block@train.csv', sep=',')

In [ ]:
df = df1.copy()

In [ ]:
display(df)

## Предобработка данных

In [ ]:
# Проверим, есть ли в датасете незаполненные значения
# Мы видим, что в столбце Cabin, Age, Embarked есть пропуски 
df.info()

In [ ]:
# Удалим столбец Cabin из-за огромного количества пропусков и столбцы PassengerId, Name, Ticket, 
# Embarked так как они не понадобятся для обучения модели
df = df.drop(['Cabin','PassengerId', 'Name', 'Ticket', 'Embarked',], axis=1)

In [ ]:
# Удалим строки, в которых пропущено значение Age
df.dropna(inplace = True)

In [ ]:
# Заменим значения в признаке Sex на 0(male) и 1(female)
df.loc[df.Sex == 'male', 'Sex'] = 0
df.loc[df.Sex == 'female', 'Sex'] = 1

In [ ]:
# Все пропуски заполнены
df.isnull().sum()

In [ ]:
# Проверим сбалансированность данных
df.Survived.value_counts().plot(kind='bar', title='Count (target)', color = ['b', 'g'])

Баланс не идеальный, но какой-то дополнительной балансировки не требуется

## Обучение моделей

Разобъем наш датасет на выборки

In [ ]:
x = df.drop('Survived', axis = 1)
y = df['Survived']
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size=0.25, random_state=14 )


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import RocCurveDisplay


def try_model(model, x_train, y_train, x_test, y_test, model_params=None):
    if model_params is None:
        model_params = dict()
    print(model.__name__)
    pipeline = make_pipeline(StandardScaler(), model(**model_params))
    pipeline.fit(x_train, y_train)
    y_pred = pipeline.predict(x_test)
    print('Accuracy:', accuracy_score(y_test, y_pred))
    print('Precision:', precision_score(y_test, y_pred))
    print('Recall:', recall_score(y_test, y_pred))
    print('F1 score:', f1_score(y_test, y_pred))
    print(y_pred)

    fpr, tpr, thresholds = roc_curve(y_test, pipeline.predict_proba(x_test)[:, 1])
    plt.plot(fpr, tpr, color="darkorange", label=f"ROC curve (area = {auc(fpr, tpr):0.3f})")
    plt.plot([0, 1], [0, 1], color="navy", linestyle="--")
    plt.legend()

    # for sklearn.__version__ of 1.0 and above
    # display = RocCurveDisplay.from_estimator(pipeline, X_test, y_test)
    # display.plot()
    # plt.show()
    return pipeline

#### Для оценки мы будем использовать метрики Accuracy, Precision, Recall, F1 score и ROC curve в совокупности и будем оценивать эффективность обучения модели по минимальному значению из полученных

#### KNeighborsClassifier

In [ ]:
try_model(KNeighborsClassifier, x_train, y_train, x_test, y_test, model_params=dict(n_neighbors=5))

Минимальное значение из полученных - 0.75

#### GaussianNB

In [ ]:
try_model(GaussianNB, x_train, y_train, x_test, y_test)

Минимальное значение из полученных - 0.71

#### LogisticRegression

In [ ]:
try_model(LogisticRegression, x_train, y_train, x_test, y_test)

Минимальное значение из полученных - 0.72

#### SVC

In [ ]:
try_model(SVC, x_train, y_train, x_test, y_test,model_params=dict(
            probability=True,
))

Минимальное значение из полученных - 0.73

## Заключение
Лучший результат показала модель KNeighborsClassifier